In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU , Bidirectional, Dropout
from keras.callbacks import ModelCheckpoint
from random import randint
import tensorflow as tf

In [ ]:
import pandas as pd

# Read the data from the CSV file
data = pd.read_csv('abba.csv')

# Concatenate all lyrics into a single string
corpus = ' '.join(data['lyrics'].dropna())

# Print the first 500 characters of the corpus just to verify
print(corpus[:500])

In [ ]:
import re

# Function to clean the text
def clean_text(text):
    # Remove Unicode characters
    cleaned_text = re.sub('[^\x00-\x7F]+', '', text)
    # Remove \r and other special characters
    cleaned_text = re.sub(r'[\r\n\t]', ' ', cleaned_text)
    # Remove extra whitespaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# Clean the corpus
corpus = clean_text(corpus)

# Save the cleaned corpus to a file
with open('cleaned_corpus.txt', 'w', encoding='utf-8') as f:
    f.write(corpus)

print("Data cleaned and saved to 'cleaned_corpus.txt'.")

In [ ]:
# Get unique characters from the corpus
unique_chars = sorted(set(corpus))

# Create encoder dictionary (character to integer)
encoder_dict = {char: i for i, char in enumerate(unique_chars)}

# Create decoder dictionary (integer to character)
decoder_dict = {i: char for i, char in enumerate(unique_chars)}

# Print encoder dictionary
print("Encoder dictionary:")
print(encoder_dict)

# Print decoder dictionary
print("\nDecoder dictionary:")
print(decoder_dict)


In [ ]:
# slice the corpus into semi-redundant sequences of 20 characters, and encode them using our dictionaries.
# Print the first 20 characters of the corpus just to verify
# sliced_corpus = cleaned_corpus[:20]
# encoder_dict(sliced_corpus)

sentence_length = 20

skip = 1
X_data = []
y_data = []

for i in range (0, len(corpus) - sentence_length, skip):
    sentence = corpus[i:i + sentence_length]
    next_char = corpus[i + sentence_length]
    X_data.append([encoder_dict[char] for char in sentence])
    y_data.append(encoder_dict[next_char])

In [ ]:
# simple check.
X_data[1], y_data[1]


In [ ]:
num_sentences = len(X_data)
print("Sliced our corpus into {0} sentences of length {1}".format(num_sentences, sentence_length))

In [ ]:
# now we need one-hot encoding
num_chars = len(unique_chars)

print("Vectorizing X and y...")
X = np.zeros((num_sentences, sentence_length, num_chars), dtype=bool)
y = np.zeros((num_sentences, num_chars), dtype=bool)
for i, sentence in enumerate(X_data):
    for t, encoded_char in enumerate(sentence):
        X[i, t, encoded_char] = 1
    y[i, y_data[i]] = 1

In [ ]:
# Define our model
print("Let's build model.")
model = Sequential()
model.add(LSTM(32, input_shape=(sentence_length, num_chars), return_sequences=True))
model.add(Bidirectional(LSTM(32, return_sequences=True)))  
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(num_chars))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary() 

In [ ]:
# Train our model! save the weights after each epoch if it's a new best for loss and save model architecture.
architecture = model.to_json()
with open('model.json', 'w') as model_file:
    model_file.write(architecture)

# Set up checkpoints, and save trained model
file_path="weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor="loss", verbose=1, save_best_only=True, mode="min")
callbacks = [checkpoint]



In [ ]:
# Train het model voor minimaal 20 epochs, En selecteer het dat weights-file die de beste resultaten geeft?
# Is er ook een weights-file waarvanje kunt zeggen dat er overfitting optreedt?

# kijk goed naar de onderstaande regel.

history = model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks)